In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
dataset = pd.read_csv('Churn_Modelling.csv')

dataset = dataset.drop(['RowNumber','CustomerId', 'Surname'], axis = 1)

le = LabelEncoder()
dataset['Gender'] = le.fit_transform(dataset['Gender'])

## oneHotEncoder on Geography field
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_encoded = ohe.fit_transform(dataset[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns = ohe.get_feature_names_out(['Geography']))

dataset = pd.concat([dataset.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

X = dataset.drop('Exited', axis = 1)
y = dataset['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
import pickle

In [12]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(le,file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe,file)

with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)        

In [30]:
# funciton to create model and try different parameters

def create_model(neurons = 32, layers = 1):
        model = Sequential()
        model.add(Dense(neurons, activation = 'relu', input_shape = (X_train.shape[1])))

        for _ in range(layers-1):
            model.add(Dense(neurons, activation = 'relu'))

        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

        return model    

In [31]:
# create a keras classifier
model = KerasClassifier(layers = 1, neurons = 32, build_fn = create_model, epochs = 50, batch_size=10, verbose = 0)

In [32]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1,2,3],
    'epochs': [50, 100]
}

In [34]:
grid  =GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 3,verbose = 1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error i

ValueError: 
All the 72 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py", line 915, in _fit
    X, y = self._initialize(X, y)
           ^

In [19]:
grid_result.best_score_, grid_result.best_params_

NameError: name 'grid_result' is not defined